In [1]:
import pandas as pd
from pathlib import Path

# 프로젝트 루트 기준 데이터 경로 설정
PROJECT_ROOT = Path.cwd().parent  # 현재 위치가 notebooks/ 라고 가정
DATA_PATH = PROJECT_ROOT / "data" / "enhanced_data.csv"

print("Data path:", DATA_PATH)

df = pd.read_csv(DATA_PATH)

print("\nShape:", df.shape)
print("\nDtypes:")
print(df.dtypes)

print("\nMissing values (count):")
print(df.isna().sum().sort_values(ascending=False))

print("\nMissing values (ratio %):")
print((df.isna().mean() * 100).sort_values(ascending=False).round(3))


Data path: c:\2nd_project\SKN21-2nd-2Team\data\enhanced_data.csv

Shape: (8000, 26)

Dtypes:
user_id                       int64
gender                       object
age                           int64
country                      object
subscription_type            object
listening_time              float64
songs_played_per_day        float64
skip_rate                   float64
device_type                  object
ads_listened_per_week         int64
offline_listening             int64
is_churned                    int64
engagement_score            float64
songs_per_minute            float64
skip_intensity              float64
skip_rate_cap               float64
ads_pressure                float64
listening_time_trend_7d     float64
login_frequency_30d           int64
days_since_last_login         int64
skip_rate_increase_7d       float64
freq_of_use_trend_14d       float64
customer_support_contact      int64
payment_failure_count         int64
promotional_email_click       int64
app_cra

After imputation - missing values (count):
engagement_score           473
songs_played_per_day       240
skip_intensity             240
listening_time             240
gender                       0
user_id                      0
subscription_type            0
skip_rate                    0
country                      0
age                          0
ads_listened_per_week        0
device_type                  0
is_churned                   0
offline_listening            0
songs_per_minute             0
skip_rate_cap                0
ads_pressure                 0
listening_time_trend_7d      0
login_frequency_30d          0
days_since_last_login        0
dtype: int64

Saved imputed data to: c:\2nd_project\SKN21-2nd-2Team\data\enhanced_data_imputed.csv


[결측치 요약]
- 원본 데이터(df) 전체 결측 개수: 1193
- 결측 처리 후(df_imputed) 전체 결측 개수: 1193

[이상치 요약 - IQR 기준]
- 이상치(한 컬럼이라도 IQR 범위 밖인 행) 포함 행 수: 2833

- 컬럼별 이상치 개수 Top 10 (IQR 기준):
  ads_listened_per_week: 1683
  songs_per_minute: 826
  payment_failure_count: 389
  skip_intensity: 170
  skip_rate: 160
  skip_rate_cap: 160
  engagement_score: 81
  age: 80
  user_id: 0
  listening_time: 0


In [10]:
# raw_data, synthetic_features, enhanced_data 간 결측치/이상치 비교

import numpy as np
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

RAW_PATH = PROJECT_ROOT / "data" / "raw_data.csv"
SYN_PATH = PROJECT_ROOT / "data" / "synthetic_features.csv"
ENH_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"

print("RAW_PATH:", RAW_PATH)
print("SYN_PATH:", SYN_PATH)
print("ENH_PATH:", ENH_PATH)

raw_df = pd.read_csv(RAW_PATH)
syn_df = pd.read_csv(SYN_PATH)
enh_df = pd.read_csv(ENH_PATH)

print("\n[기본 정보]")
for name, d in [("raw_data", raw_df), ("synthetic_features", syn_df), ("enhanced_data", enh_df)]:
    print(f"- {name}: shape={d.shape}, 전체 결측 개수={int(d.isna().sum().sum())}")


def summarize_outliers(name, df):
    """IQR 기준 이상치 행 수 및 컬럼별 이상치 개수 요약"""
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
    outlier_mask_global = np.zeros(len(df), dtype=bool)
    outlier_counts_per_col = {}

    for col in numeric_cols:
        series = df[col].dropna()
        if series.empty:
            continue
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        if iqr == 0:
            continue
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        mask = (df[col] < lower) | (df[col] > upper)
        count = int(mask.sum())
        outlier_counts_per_col[col] = count
        outlier_mask_global |= mask

    total_outlier_rows = int(outlier_mask_global.sum())

    print(f"\n[{name} - 이상치 요약 (IQR 기준)]")
    print(f"- 이상치(한 컬럼이라도 범위 밖인 행 수): {total_outlier_rows}")

    print("- 컬럼별 이상치 개수 Top 5:")
    for col, cnt in sorted(outlier_counts_per_col.items(), key=lambda x: -x[1])[:5]:
        print(f"  {col}: {cnt}")


summarize_outliers("raw_data", raw_df)
summarize_outliers("synthetic_features", syn_df)
summarize_outliers("enhanced_data", enh_df)


RAW_PATH: c:\2nd_project\SKN21-2nd-2Team\data\raw_data.csv
SYN_PATH: c:\2nd_project\SKN21-2nd-2Team\data\synthetic_features.csv
ENH_PATH: c:\2nd_project\SKN21-2nd-2Team\data\enhanced_data_clean.csv

[기본 정보]
- raw_data: shape=(8000, 12), 전체 결측 개수=480
- synthetic_features: shape=(8000, 10), 전체 결측 개수=0
- enhanced_data: shape=(8000, 26), 전체 결측 개수=0

[raw_data - 이상치 요약 (IQR 기준)]
- 이상치(한 컬럼이라도 범위 밖인 행 수): 1878
- 컬럼별 이상치 개수 Top 5:
  ads_listened_per_week: 1683
  skip_rate: 160
  age: 80
  user_id: 0
  listening_time: 0

[synthetic_features - 이상치 요약 (IQR 기준)]
- 이상치(한 컬럼이라도 범위 밖인 행 수): 389
- 컬럼별 이상치 개수 Top 5:
  payment_failure_count: 389
  user_id: 0
  listening_time_trend_7d: 0
  login_frequency_30d: 0
  days_since_last_login: 0

[enhanced_data - 이상치 요약 (IQR 기준)]
- 이상치(한 컬럼이라도 범위 밖인 행 수): 175
- 컬럼별 이상치 개수 Top 5:
  skip_intensity: 175
  user_id: 0
  age: 0
  listening_time: 0
  songs_played_per_day: 0


In [8]:
# enhanced_data에서 결측치 + 이상치 처리 후 clean CSV 생성

import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
ENH_PATH = PROJECT_ROOT / "data" / "enhanced_data.csv"

enh = pd.read_csv(ENH_PATH)
print("원본 enhanced_data shape:", enh.shape)
print("원본 전체 결측 개수:", int(enh.isna().sum().sum()))

# 1) 결측치 처리
# 원본 수치형에서 결측이 있는 컬럼: listening_time, songs_played_per_day
missing_base_cols = ["listening_time", "songs_played_per_day"]

for col in missing_base_cols:
    if col in enh.columns:
        median_val = enh[col].median()
        enh[col] = enh[col].fillna(median_val)
        print(f"{col}: 결측치를 median({median_val:.3f})으로 대체")

# 파생 컬럼은 다시 계산해서 NaN 제거 (일관성 유지)
if set(["listening_time", "songs_played_per_day"]).issubset(enh.columns):
    enh["engagement_score"] = enh["listening_time"] * enh["songs_played_per_day"]
    enh["skip_intensity"] = enh["skip_rate"] * enh["songs_played_per_day"]
    print("engagement_score, skip_intensity 파생 컬럼 재계산 완료")

print("\n결측 처리 후 전체 결측 개수:", int(enh.isna().sum().sum()))

# 2) 이상치 처리 (IQR 기반 winsorizing)
# user_id, is_churned는 대상에서 제외
numeric_cols = enh.select_dtypes(include=["int64", "float64"]).columns
numeric_cols = [c for c in numeric_cols if c not in ["user_id", "is_churned"]]

print("\n[IQR 기반 이상치 winsorizing 적용 컬럼 수]", len(numeric_cols))

for col in numeric_cols:
    series = enh[col].dropna()
    if series.empty:
        continue
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        continue
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    enh[col] = enh[col].clip(lower, upper)

# 이상치 처리 후 IQR 기준 이상치 행 수 재계산
from collections import defaultdict

outlier_mask_global = np.zeros(len(enh), dtype=bool)
outlier_counts_per_col = defaultdict(int)

for col in numeric_cols:
    series = enh[col].dropna()
    if series.empty:
        continue
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        continue
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    mask = (enh[col] < lower) | (enh[col] > upper)
    cnt = int(mask.sum())
    outlier_counts_per_col[col] = cnt
    outlier_mask_global |= mask

total_outlier_rows = int(outlier_mask_global.sum())
print("\n[IQR 기준 이상치 요약 - 처리 후]")
print("- 이상치(한 컬럼이라도 범위 밖인 행 수):", total_outlier_rows)
print("- 컬럼별 이상치 개수 Top 5:")
for col, cnt in sorted(outlier_counts_per_col.items(), key=lambda x: -x[1])[:5]:
    print(f"  {col}: {cnt}")

# 3) clean CSV 저장
CLEAN_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"
enh.to_csv(CLEAN_PATH, index=False)
print(f"\nclean 데이터 저장 완료: {CLEAN_PATH}")


원본 enhanced_data shape: (8000, 26)
원본 전체 결측 개수: 1193
listening_time: 결측치를 median(154.000)으로 대체
songs_played_per_day: 결측치를 median(50.000)으로 대체
engagement_score, skip_intensity 파생 컬럼 재계산 완료

결측 처리 후 전체 결측 개수: 0

[IQR 기반 이상치 winsorizing 적용 컬럼 수] 20

[IQR 기준 이상치 요약 - 처리 후]
- 이상치(한 컬럼이라도 범위 밖인 행 수): 0
- 컬럼별 이상치 개수 Top 5:
  age: 0
  listening_time: 0
  songs_played_per_day: 0
  skip_rate: 0
  ads_listened_per_week: 0

clean 데이터 저장 완료: c:\2nd_project\SKN21-2nd-2Team\data\enhanced_data_clean.csv


In [16]:
# enhanced_data_clean.csv 기준 RandomForest 성능 테스트

import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report

PROJECT_ROOT = Path.cwd().parent
CLEAN_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"

df_rf = pd.read_csv(CLEAN_PATH)
print("clean 데이터 shape:", df_rf.shape)

# 타깃과 피처 분리
y = df_rf["is_churned"]

# 수치형 피처만 사용 (범주형 인코딩 없이 빠르게 성능 확인)
feature_cols = df_rf.select_dtypes(include=["int64", "float64"]).columns
feature_cols = [c for c in feature_cols if c not in ["user_id", "is_churned"]]
X = df_rf[feature_cols]

print("사용 피처 수:", len(feature_cols))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    class_weight="balanced",
    n_jobs=-1,
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
if hasattr(rf, "predict_proba"):
    y_proba = rf.predict_proba(X_test)[:, 1]
else:
    # predict_proba가 없을 경우 decision_function 사용 (예비용)
    from sklearn.metrics import roc_curve
    scores = rf.decision_function(X_test)
    # 점수를 0~1로 스케일링
    y_proba = (scores - scores.min()) / (scores.max() - scores.min())

f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print("\n[RandomForest 성능 - enhanced_data_clean 기준]")
print("F1:", round(f1, 4))
print("AUC:", round(auc, 4))
print("\nClassification report:\n", classification_report(y_test, y_pred))


clean 데이터 shape: (8000, 26)
사용 피처 수: 20

[RandomForest 성능 - enhanced_data_clean 기준]
F1: 0.5574
AUC: 0.7933

Classification report:
               precision    recall  f1-score   support

           0       0.83      0.94      0.88      1186
           1       0.73      0.45      0.56       414

    accuracy                           0.81      1600
   macro avg       0.78      0.70      0.72      1600
weighted avg       0.80      0.81      0.80      1600



In [11]:
# FE 포함 vs 제외 RF 성능 비교 실험

import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

PROJECT_ROOT = Path.cwd().parent
CLEAN_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"

df_cmp = pd.read_csv(CLEAN_PATH)

# 공통: 타깃
y = df_cmp["is_churned"]

# FE 5개 (문서 기준)
fe_cols = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
    "ads_pressure",
]

# 수치형 전체 (user_id, target 제외)
all_num_cols = df_cmp.select_dtypes(include=["int64", "float64"]).columns
all_num_cols = [c for c in all_num_cols if c not in ["user_id", "is_churned"]]

# 1) FE 포함 (현재 우리가 쓰는 구성과 비슷)
X_all = df_cmp[all_num_cols]

# 2) FE 5개 제외 (원본 수치형 + 시계열/고객접점만)
cols_wo_fe = [c for c in all_num_cols if c not in fe_cols]
X_wo_fe = df_cmp[cols_wo_fe]

print("전체 수치형 피처 수 (FE 포함):", len(all_num_cols))
print("FE 제외 피처 수:", len(cols_wo_fe))


def eval_rf(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1,
    )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_proba = rf.predict_proba(X_test)[:, 1]
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    print(f"\n[{name}] F1={f1:.4f}, AUC={auc:.4f}")
    return f1, auc

f1_all, auc_all = eval_rf(X_all, y, "FE 포함 (모든 수치형)")
f1_wo, auc_wo = eval_rf(X_wo_fe, y, "FE 5개 제외")


전체 수치형 피처 수 (FE 포함): 20
FE 제외 피처 수: 15

[FE 포함 (모든 수치형)] F1=0.5574, AUC=0.7933

[FE 5개 제외] F1=0.5810, AUC=0.7946


In [12]:
# [추가 실험] 원본 enhanced_data (결측 처리/윈저라이징 없이)에서 FE 포함 vs 제외 비교

import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

PROJECT_ROOT = Path.cwd().parent
ENH_PATH = PROJECT_ROOT / "data" / "enhanced_data.csv"

df_raw_enh = pd.read_csv(ENH_PATH)
print("원본 enhanced_data shape:", df_raw_enh.shape)
print("원본 전체 결측 개수:", int(df_raw_enh.isna().sum().sum()))

# 타깃
y_full = df_raw_enh["is_churned"]

# FE 5개
fe_cols = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
    "ads_pressure",
]

# 수치형 전체 (user_id, target 제외)
all_num_cols = df_raw_enh.select_dtypes(include=["int64", "float64"]).columns
all_num_cols = [c for c in all_num_cols if c not in ["user_id", "is_churned"]]

# 1) FE 포함
X_all_full = df_raw_enh[all_num_cols]

# 2) FE 제외
cols_wo_fe = [c for c in all_num_cols if c not in fe_cols]
X_wo_fe_full = df_raw_enh[cols_wo_fe]

print("원본 수치형 피처 수 (FE 포함):", len(all_num_cols))
print("원본 FE 제외 피처 수:", len(cols_wo_fe))


def eval_rf_dropna(X_full, y_full, name):
    # sklearn은 NaN을 직접 처리 못하므로, 사용 피처 기준으로 NaN 행은 드롭
    mask = X_full.notna().all(axis=1)
    X = X_full.loc[mask].copy()
    y = y_full.loc[mask].copy()
    print(f"{name} - 사용 행 수: {X.shape[0]} (드롭된 행: {len(y_full) - len(y)})")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1,
    )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_proba = rf.predict_proba(X_test)[:, 1]

    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    print(f"  → F1={f1:.4f}, AUC={auc:.4f}\n")
    return f1, auc

f1_all_raw, auc_all_raw = eval_rf_dropna(X_all_full, y_full, "[원본] FE 포함")
f1_wo_raw, auc_wo_raw = eval_rf_dropna(X_wo_fe_full, y_full, "[원본] FE 제외")


원본 enhanced_data shape: (8000, 26)
원본 전체 결측 개수: 1193
원본 수치형 피처 수 (FE 포함): 20
원본 FE 제외 피처 수: 15
[원본] FE 포함 - 사용 행 수: 7527 (드롭된 행: 473)
  → F1=0.6244, AUC=0.8207

[원본] FE 제외 - 사용 행 수: 7527 (드롭된 행: 473)
  → F1=0.6338, AUC=0.8280



In [13]:
# FE 완전 제거 + 시계열/고객접점 유지 상태에서
# [원본 enhanced_data (결측 drop)] vs [enhanced_data_clean (결측/이상치 처리)] RF 성능 비교

import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

PROJECT_ROOT = Path.cwd().parent
ENH_PATH = PROJECT_ROOT / "data" / "enhanced_data.csv"
CLEAN_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"

raw_enh = pd.read_csv(ENH_PATH)
clean_enh = pd.read_csv(CLEAN_PATH)

print("raw_enh shape:", raw_enh.shape, " / 전체 결측:", int(raw_enh.isna().sum().sum()))
print("clean_enh shape:", clean_enh.shape, " / 전체 결측:", int(clean_enh.isna().sum().sum()))

# 타깃
y_raw = raw_enh["is_churned"]
y_clean = clean_enh["is_churned"]

# FE 5개는 완전히 제거
fe_cols = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
    "ads_pressure",
]

# 공통으로 쓸 수치형 피처 목록 (user_id, is_churned, FE 제외)
num_cols_raw = raw_enh.select_dtypes(include=["int64", "float64"]).columns
num_cols_raw = [c for c in num_cols_raw if c not in ["user_id", "is_churned"] + fe_cols]

num_cols_clean = clean_enh.select_dtypes(include=["int64", "float64"]).columns
num_cols_clean = [c for c in num_cols_clean if c not in ["user_id", "is_churned"] + fe_cols]

# 양쪽에서 공통으로 존재하는 컬럼만 사용 (혹시라도 타입/이름 차이 방지)
common_cols = sorted(list(set(num_cols_raw) & set(num_cols_clean)))

X_raw_full = raw_enh[common_cols]
X_clean_full = clean_enh[common_cols]

print("공통 수치형 피처 수 (FE 제거 후):", len(common_cols))
print("공통 피처 예시:", common_cols[:10])


def eval_rf_with_optional_dropna(X_full, y_full, name, dropna=True):
    # dropna=True이면 해당 피처 기준 NaN 있는 행 제거 (원본용)
    if dropna:
        mask = X_full.notna().all(axis=1)
        X = X_full.loc[mask].copy()
        y = y_full.loc[mask].copy()
        dropped = len(y_full) - len(y)
    else:
        X = X_full.copy()
        y = y_full.copy()
        dropped = 0

    print(f"{name} - 사용 행 수: {X.shape[0]} (드롭된 행: {dropped})")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1,
    )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_proba = rf.predict_proba(X_test)[:, 1]

    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    print(f"  → F1={f1:.4f}, AUC={auc:.4f}\n")
    return f1, auc

# 1) 원본 enhanced_data: 결측 drop 기반
f1_raw_fe_off, auc_raw_fe_off = eval_rf_with_optional_dropna(
    X_raw_full, y_raw, "[원본] FE 제거 + 결측 drop", dropna=True
)

# 2) clean 데이터: 결측/이상치 처리 후, drop 없이 사용
f1_clean_fe_off, auc_clean_fe_off = eval_rf_with_optional_dropna(
    X_clean_full, y_clean, "[clean] FE 제거 + 결측/이상치 처리", dropna=False
)


raw_enh shape: (8000, 26)  / 전체 결측: 1193
clean_enh shape: (8000, 26)  / 전체 결측: 0
공통 수치형 피처 수 (FE 제거 후): 15
공통 피처 예시: ['ads_listened_per_week', 'age', 'app_crash_count_30d', 'customer_support_contact', 'days_since_last_login', 'freq_of_use_trend_14d', 'listening_time', 'listening_time_trend_7d', 'login_frequency_30d', 'offline_listening']
[원본] FE 제거 + 결측 drop - 사용 행 수: 7527 (드롭된 행: 473)
  → F1=0.6151, AUC=0.8287

[clean] FE 제거 + 결측/이상치 처리 - 사용 행 수: 8000 (드롭된 행: 0)
  → F1=0.5664, AUC=0.7957



## 🔚 Preprocessing v2 결론 정리

- **최종 기준 데이터**
  - **EDA 기준**: `data/enhanced_data_clean.csv` (8000 × 26)
    - 구성: 원본 12개 컬럼(`user_id`, `gender`, `age`, `country`, `subscription_type`, `listening_time`, `songs_played_per_day`, `skip_rate`, `device_type`, `ads_listened_per_week`, `offline_listening`, `is_churned`)  
      + 기존 FE 5개(`engagement_score`, `songs_per_minute`, `skip_intensity`, `skip_rate_cap`, `ads_pressure`)  
      + 시계열 피처 5개(`listening_time_trend_7d`, `login_frequency_30d`, `days_since_last_login`, `skip_rate_increase_7d`, `freq_of_use_trend_14d`)  
      + 고객접점 피처 4개(`customer_support_contact`, `payment_failure_count`, `promotional_email_click`, `app_crash_count_30d`)
    - 역할: 정제된 전체 구조/분포 및 정제 전·후 비교를 위한 **EDA/리포트용 데이터**
  - **모델 입력 기준**: `data/enhanced_data_clean_model.csv` (8000 × 21)
    - 구성: `enhanced_data_clean.csv`에서 **FE 5개 컬럼(`engagement_score`, `songs_per_minute`, `skip_intensity`, `skip_rate_cap`, `ads_pressure`)만 제거**한 버전  
      → 원본 수치형 6 + 시계열 5 + 고객접점 4 = **총 15개 수치 피처**를 모델 입력으로 사용

- **결측치 처리 규칙 (`enhanced_data_clean.csv` 생성)**
  - `listening_time`, `songs_played_per_day`: 각 컬럼 **median**으로 대체  
    (→ 두 컬럼은 최종 15개 수치 피처의 기반이 되는 핵심 활동 지표이므로, 결측 시 중앙값으로 보수적으로 대체)
  - `payment_failure_count`, `app_crash_count_30d`: NaN → **0회**로 간주
  - `customer_support_contact`, `promotional_email_click`: NaN → **False(이벤트 없음)** 로 간주
  - 그 외 시계열 트렌드/변화율 컬럼(`listening_time_trend_7d`, `freq_of_use_trend_14d`, `skip_rate_increase_7d` 등)은 합성 단계에서 이미 결측 없음 확인

- **이상치 처리 규칙 (`enhanced_data_clean.csv` 생성)**
  - 대상: 모든 수치형 컬럼(단, `user_id`, `is_churned` 제외)
  - 방법: IQR 기준으로 **[Q1 − 1.5×IQR, Q3 + 1.5×IQR]** 범위를 벗어나는 값은 해당 경계값으로 **clip(winsorizing)**

- **RandomForest 기준 성능 요약 (FE 제거 + 시계열/고객접점 포함, 최종 15개 수치 피처)**
  - [원본 `enhanced_data.csv` + 결측 행 drop, 7,527행 사용]  
    - F1 **0.6151**, AUC **0.8287**  
    - 결측/극단값이 있는 행을 제거하여 **예측이 쉬운 샘플 위주의 상한선(upper bound) 성능**으로 해석
  - [`enhanced_data_clean.csv` (결측/이상치 처리, 8,000행 전부 사용) + `enhanced_data_clean_model.csv` (FE 제거 후 15개 피처 사용)]  
    - F1 **0.5664**, AUC **0.7957**  
    - 정제 과정으로 인해 F1 약 **−0.049p(≈ −7.9%)**, AUC 약 **−0.033p(≈ −4.0%)** 감소  
    - 대신 **모든 유저를 사용**하고, 이상치 완화로 **모델의 일반화 안정성**을 확보

- **최종 결정**
  - 모델 학습/튜닝 및 배포용 파이프라인에서는 **FE 5개(`engagement_score`, `songs_per_minute`, `skip_intensity`, `skip_rate_cap`, `ads_pressure`)를 사용하지 않고**,  
    원본 수치형 6 + 시계열 5 + 고객접점 4 = **15개 수치 피처만 입력으로 사용**한다.
  - EDA 노트북(`eda_add.ipynb`)에서는 `enhanced_data.csv` / `enhanced_data_clean.csv`를 활용해  
    결측/이상치 처리 전·후의 분포 차이와 시계열/고객접점 피처의 효과를 설명한다.
  - 모델 성능 비교 및 하이퍼파라미터 튜닝은 **`enhanced_data_clean_model.csv` + 15개 피처를 공통 기준**으로 진행한다.

- **모델용 축소 데이터 (FE 제거 CSV)**
  - 파일: `data/enhanced_data_clean_model.csv`
  - 구성: `enhanced_data_clean.csv`에서 **FE 5개 컬럼만 제거**한 버전
  - 제거된 컬럼: `engagement_score`, `songs_per_minute`, `skip_intensity`, `skip_rate_cap`, `ads_pressure`
  - 용도: 최종 모델 학습/튜닝 및 모델 간 성능 비교를 위한 공통 입력 데이터로 사용


In [17]:
# 모델용 CSV: FE 5개를 제거한 버전 저장

import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
CLEAN_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean.csv"
MODEL_PATH = PROJECT_ROOT / "data" / "enhanced_data_clean_model.csv"

fe_cols = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
    "ads_pressure",
]

df_full = pd.read_csv(CLEAN_PATH)
print("원본 clean shape:", df_full.shape)

keep_df = df_full.drop(columns=fe_cols, errors="ignore")
print("모델용 shape (FE 5개 제거 후):", keep_df.shape)

keep_df.to_csv(MODEL_PATH, index=False)
print(f"모델용 CSV 저장 완료: {MODEL_PATH}")


원본 clean shape: (8000, 26)
모델용 shape (FE 5개 제거 후): (8000, 21)
모델용 CSV 저장 완료: c:\2nd_project\SKN21-2nd-2Team\data\enhanced_data_clean_model.csv
